In [1]:
%load_ext autoreload
%autoreload 2

from mint.model.transformer import Transformer, TransformerConfig
from mint.common import create_config, to_dict
from mint.trainer import Trainer, TrainerConfig
import os
from mint.translator import Translator
from mint.dataset import Dataset
from mint.tokenizer import Tokenizer

In [2]:
DATASET_PATH = "../datasets/en_sk/"

source_tokenizer = Tokenizer.load("../datasets/en_sk/source_tokenizer/")
target_tokenizer = Tokenizer.load("../datasets/en_sk/target_tokenizer/")

dataset = Dataset.load(DATASET_PATH)


In [3]:
config = create_config(TransformerConfig())
config.glob.d_model = 512
config.glob.n_heads = 8
config.glob.max_seq_len = 128 + 1
config.glob.d_feedforward = 2048
config.glob.p_dropout = 0.1

config.encoder_config.n_blocks = 10
config.encoder_config.vocab_size = 10000 + 1
config.decoder_config.n_blocks = 10
config.decoder_config.vocab_size = 10000 + 1

model = Transformer.load("../pretrained/model")


print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

{'n_blocks': 10, 'vocab_size': 10001, 'transformer_block_config': {'d_model': 512, 'd_feedforward': 2048, 'p_dropout': 0.1, 'attention_config': {'n_heads': 8, 'd_model': 512, 'max_seq_len': 129, 'context_window': None}}, 'embedding_config': {'vocab_size': 10001, 'd_model': 512, 'max_seq_len': 129, 'learnable_positional_embeddings': True}}


/home/mharag/fit/zpja/mint/model/transformer.py:179: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(path, "weights.pt")))


Number of parameters: 126475776


In [ ]:
trainer_config = create_config(TrainerConfig())
trainer_config.logger_config.experiment_name = "exp2"
trainer_config.warmup_steps = 3000
trainer_config.learning_rate = 1e-4
trainer_config.use_cuda = True
trainer_config.max_steps_per_epoch = 100000 // 32 # too big dataset to run locally
trainer_config.max_steps_per_validation = 1000 // 32
trainer = Trainer(model, dataset, **to_dict(trainer_config), source_tokenizer=source_tokenizer, target_tokenizer=target_tokenizer)
trainer.train(10)

Train 1/10 avg loss: 0.7626:   0%|          | 13/3125 [00:09<38:07,  1.36it/s]

In [17]:
translator = Translator(model, source_tokenizer, target_tokenizer)

translator.translate("Good evening", max_length=128)

['Dobré večera<|endoftext|>']